In [ ]:
%matplotlib inline
import matplotlib as plt
from ipywidgets import widgets

### Normal distribution

### Bernoulli distribution

### Geometric distribution

### Binomial distribution

### Poisson distribution

### Negative binomial distribution